#### 1. Install TensorFlow Object Detection API

In [ ]:
%tensorflow_version 1.x

In [ ]:
#Clone githun repository
!git clone https://github.com/tensorflow/models

In [ ]:
#!cp /content/models/research/object_detection/models/ssd_mobilenet_v1_feature_extractor.py .

In [ ]:
#!cp ssd_mobilenet_v1_feature_extractor.py models/research/object_detection/models/

In [ ]:
#Upgrade pip package
!pip install --upgrade pip

In [ ]:
%%bash
cd models/research
# Compile protos.
protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
cp object_detection/packages/tf1/setup.py .
python -m pip install --use-feature=2020-resolver .

In [ ]:
#Check if Object Detection API is installed
!pip list | grep object-detection

#### Check Anchor Boxes

In [ ]:
import warnings
warnings.filterwarnings("ignore")

Reference -> https://gist.github.com/david-macleod/60cb2b00616562b9b613bea618f9956b

In [ ]:
import sys
sys.path.append('/content/models/research')
sys.path.append('/content/models/research/slim')

In [ ]:
import tensorflow as tf 
from object_detection.anchor_generators.multiple_grid_anchor_generator import create_ssd_anchors
from object_detection.models.ssd_mobilenet_v1_feature_extractor_tf1_test import SsdMobilenetV1FeatureExtractorTest
import matplotlib.pyplot as plt
from matplotlib import patches
import numpy as np

Understand feature map shapes

In [ ]:
def get_feature_map_shapes(image_height, image_width, depth=1, num_layers=6):
    """
    :param image_height: height in pixels
    :param image_width: width in pixels
    :returns: list of tuples containing feature map resolutions
    """
    
    feature_extractor = SsdMobilenetV1FeatureExtractorTest()._create_feature_extractor(
        depth_multiplier=depth,
        pad_to_multiple=1,
        num_layers=num_layers
    )

    image_batch_tensor = tf.zeros([1, image_height, image_width, 1])

    return [tuple(feature_map.get_shape().as_list()[1:3])
            for feature_map in feature_extractor.extract_features(image_batch_tensor)]

In [ ]:
tf.reset_default_graph()
feature_map_size = get_feature_map_shapes(128,128, depth=0.25, num_layers=5)
feature_map_size

In [ ]:
feature_map_size = [(4, 4), (2, 2), (1, 1), (1, 1),(1,1)]

Anchor Boxes for each feature map

In [ ]:
def get_feature_map_anchor_boxes(feature_map_shape_list, **anchor_kwargs):
    """
    :param feature_map_shape_list: list of tuples containing feature map resolutions
    :returns: dict with feature map shape tuple as key and list of [ymin, xmin, ymax, xmax] box co-ordinates
    """
    anchor_generator = create_ssd_anchors(**anchor_kwargs)

    anchor_box_lists = anchor_generator.generate(feature_map_shape_list)
    
    feature_map_boxes = {}
    feature_map_list = []

    with tf.Session() as sess:
        for shape, box_list in zip(feature_map_shape_list, anchor_box_lists):
            feature_map_boxes[shape] = sess.run(box_list.data['boxes'])
            feature_map_list.append(sess.run(box_list.data['boxes']))
            
    return feature_map_boxes, feature_map_list

In [ ]:
anchor_boxes, anchor_list = get_feature_map_anchor_boxes(feature_map_size, 
                                            min_scale= 0.2, 
                                            max_scale= 0.95,
                                            num_layers=len(feature_map_size),
                                            aspect_ratios= [1.0])   

In [ ]:
anchor_list[1][41]

In [ ]:
total_anchors = 0
for key in anchor_boxes.keys():
    print("Grid Size: ", key, "RoIs", anchor_boxes[key].shape, "=", anchor_boxes[key].shape[0])
    total_anchors += anchor_boxes[key].shape[0]
print('Total Anchors', total_anchors)

In [ ]:
anchor_list

In [ ]:
#anchor_boxes

Drawing anchor boxes

In [ ]:
def draw_boxes(boxes, figsize, nrows, ncols, grid=(0,0)):

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize) 

    for ax, box in zip(axes.flat, boxes):
        ymin, xmin, ymax, xmax = box
        ax.add_patch(patches.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, 
                                fill=False, edgecolor='red', lw=2))

        # add gridlines to represent feature map cells
        ax.set_xticks(np.linspace(0, 1, grid[0] + 1), minor=True)
        ax.set_yticks(np.linspace(0, 1, grid[1] + 1), minor=True)
        ax.grid(True, which='minor', axis='both')
              
    fig.tight_layout()
    
    return fig

In [ ]:
draw_boxes(anchor_boxes[(4,4)] , figsize=(9,9), nrows=10, ncols=10)

In [ ]:
draw_boxes([anchor_list[1][41]] , figsize=(9,9), nrows=2, ncols=2   )